In [1]:
import pandas as pd
from os import listdir
from os.path import isfile, join
import numpy as np
import copy
import numpy as np
import random
import copy
import itertools
import time

In [4]:
negative_features = ['Двигатель не мощный', 'Двигатель не надежный', 'Двигатель ест масло',
                    'Двигатель не экономичный', 'Машина не экономичная', 'Большой расход',
                    'Переключает медленно', 'Кузов гниет', 'Кузов склонен к коррозии',
                    'Машина не вместительная', 'Машина не очень вместительная', 
                    'Не просторно', 'Немного места', 'Плохой обзор', 'Слабый обзор',
                    'Маленькие зеркала','Маленький салон','Не удобный салон',
                    'Слабая шумоизоляция', 'Все слышно', 'Маленький багажник',
                    'Плохая управляемость','Слабая динамика','Авто медленное',
                    'Авто очень медленное','Динамики не хватает','Медленный разгон',
                    'Дорогие детали', 'Недешевые детали', 'Бракованные запчасти', 
                     'Дорогие запчасти','Недешевые запчасти','Дорогое обслуживание',
                     'Недешевое обслуживание', 'Сложное управление']

In [5]:
tech_features = ['Двигатель мощный', 'Двигатель надежный', 'Двигатель не ест масло',
'Двигатель экономичный',
'Двигатель не мощный',
'Двигатель не надежный',
'Двигатель ест масло',
'Двигатель не экономичный',
'Машина экономичная',
'Небольшой расход',
'Маленький расход',
'Машина не экономичная',
'Большой расход',
'Переключает быстро',
'Переключает медленно',
'Подвеска мягкая',
'Подвеска высокая',
'Подвеска жесткая',
'Подвеска низкая']
body_features = ['Крепкий кузов',
                 'Кузов не гниет',
                 'Кузов не склонен к коррозии',
                 'Кузов гниет',
                 'Кузов склонен к коррозии',
                 'Машина вместительная',
                 'Машина очень вместительная',
                 'Просторно',
                 'Много места',
                 'Машина не вместительная',
                 'Машина не очень вместительная',
                 'Не просторно',
                 'Немного места',
                 'Отличный обзор',
                 'Большие зеркала',
                 'Плохой обзор',
                 'Слабый обзор',
                 'Маленькие зеркала',
                 'Салон супер',
                 'Удобный салон',
                 'Печка жарит',
                 'Кондиционер морозит',
                 'Шумоизоляция на уровне',
                 'Классная шумоизоляция',
                 'Замечательная шумоизоляция',
                 'Дорогу не слышно',
                 'Улицу не слышно',
                 'Как в танке',
                 'Маленький салон',
                 'Не удобный салон',
                 'Слабая шумоизоляция',
                 'Все слышно',
                 'Большой багажник',
                 'Маленький багажник'
]
driving_features = [
    'Отличное управление',
    'Легкое управление',
    'Неплохая управляемость',
    'Сложное управление',
    'Плохая управляемость',
    'Ураганная динамика',
    "Хорошая динамика",
    'Динамичное авто',
    'Машина очень быстрая',
    'Очень шустрая',
    'Шустрая',
    'Динамика супер',
    'Динамики хватает',
    'Динамики за глаза',
    'Разгон быстрый',
    'Слабая динамика',
    'Авто медленное',
    'Авто очень медленное',
    'Динамики не хватает',
    'Медленный разгон'
]
service_features = [
    'Детали не дорогие',
    'Дешевые детали',
    'Дешевые запчасти',
    'Недорогие запчасти',
    'Дорогие детали',
    'Недешевые детали',
    'Бракованные запчасти',
    'Дорогие запчасти',
    'Недешевые запчасти',
    'Недорогое обслуживание',
    'Дешевое обслуживание',
    'Дорогое обслуживание',
    'Недешевое обслуживание']

In [6]:
import re

import numpy as np

from asts import ast


def clear_text(text, lowerize=True):

    pat = re.compile(r'[^A-Za-z0-9 \-\n\r.,;!?А-Яа-я]+')
    cleared_text = re.sub(pat, ' ', text)

    if lowerize:
        cleared_text = cleared_text.lower()

    tokens = cleared_text.split()
    return tokens


def make_substrings(tokens, k=4):

    for i in range(max(len(tokens) - k + 1, 1)):
        yield ' '.join(tokens[i:i + k])


def get_relevance_matrix(texts, strings):

    matrix = np.empty((0, len(strings)), float)
    prepared_text_tokens = [clear_text(t) for t in texts]

    prepared_string_tokens = [clear_text(s) for s in strings]
    prepared_strings = [' '.join(t) for t in prepared_string_tokens]

    for text_tokens in prepared_text_tokens:
        ast1 = ast.AST(list(make_substrings(text_tokens)))
        row = np.array([ast1.score(s) for s in prepared_strings])
        matrix = np.append(matrix, [row], axis=0)

    return matrix

def save_matrix(matrix):
    np.savetxt("relevance_matrix.txt", matrix)


In [7]:
reviews = []
path2dir = '/Users/denissurin/Диплом Многокластерный анализ/''Более 35тыс отзывов '
file_paths = [f for f in listdir(path2dir) if isfile(join(path2dir, f))]
for file_path in file_paths[:1000]:
    file = open(path2dir + '/' + file_path, 'r')
    reviews.append(file.read().replace('\n', ' '))

In [8]:
%%time
relevance_matrix_service = get_relevance_matrix(reviews, service_features)
relevance_matrix_tech = get_relevance_matrix(reviews, tech_features)
relevance_matrix_driving = get_relevance_matrix(reviews, driving_features)
relevance_matrix_body = get_relevance_matrix(reviews, body_features)

CPU times: user 17min 16s, sys: 6.87 s, total: 17min 23s
Wall time: 17min 26s


In [9]:
threshold = 0.2

In [10]:
%%time
weights = np.zeros(len(reviews))
for i in range(len(reviews)):
    s = (relevance_matrix_service[i, :] > threshold).sum() 
    t = (relevance_matrix_tech[i, :] > threshold).sum()
    d = (relevance_matrix_driving[i, :] > threshold).sum()
    b = (relevance_matrix_body[i, :] > threshold).sum()
    weights[i] = (s + t + b + d)/4

CPU times: user 22.3 ms, sys: 1.27 ms, total: 23.6 ms
Wall time: 22.7 ms


In [11]:
%%time
data_matrix = np.zeros([len(service_features), len(tech_features), 
                        len(driving_features), len(body_features)])

def check(text_id, service_id, tech_id, driving_id, body_id):
    service = relevance_matrix_service[text_id, service_id]
    if service_features[service_id] in negative_features:
        service = service * -1
    tech = relevance_matrix_tech[text_id, tech_id]
    if tech_features[tech_id] in negative_features:
        tech = tech * -1
    driving = relevance_matrix_driving[text_id, driving_id]
    if driving_features[driving_id] in negative_features:
        driving = driving * -1
    body = relevance_matrix_body[text_id, body_id]
    if body_features[body_id] in negative_features:
        body = body * -1
    return service * tech * driving * body
n = len(reviews)
for text_id in range(n):
    if weights[text_id] == 0:
        continue
    if text_id % 100 == 0:
        print(text_id)
    data_matrix_cur = np.zeros([len(service_features), len(tech_features), 
                        len(driving_features), len(body_features)])
    for service_id in range(len(service_features)):
        for tech_id in range(len(tech_features)):
            for driving_id in range(len(driving_features)):
                for body_id in range(len(body_features)):
                    cur_res = check(text_id, service_id, tech_id, driving_id, body_id)
                    data_matrix_cur[service_id, tech_id, driving_id, body_id] = cur_res
    data_matrix_cur/=weights[text_id]#потом все умножаем на 1000, чтобы числа были больше
    data_matrix += data_matrix_cur

0
100
200
300
400
500
600
700
800
900
CPU times: user 9min 47s, sys: 1.75 s, total: 9min 49s
Wall time: 9min 51s


In [308]:
for w in weights:
    print(w)

10.5
7.25
3.5
4.0
3.75
7.0
6.0
5.0
2.5
4.0
6.75
6.5
0.0
8.0
4.0
4.5
7.75
6.0
5.5
3.25
7.5
2.25
5.0
8.25
3.75
2.25
1.25
7.0
5.5
4.5
1.0
2.0
11.0
4.75
3.75
6.75
0.5
4.5
2.75
3.5
7.25
4.5
3.5
3.25
1.0
6.5
2.5
13.0
9.75
7.0
6.0
2.0
7.0
9.25
6.75
1.5
4.5
6.75
10.5
6.0
7.25
1.5
2.5
7.75
4.0
2.0
6.5
4.75
8.5
0.5
6.75
6.5
1.25
2.0
4.0
6.0
5.25
7.25
9.0
6.0
6.0
8.5
6.75
11.5
9.0
5.75
1.5
7.0
3.5
5.75
7.25
2.5
7.25
1.75
3.25
4.0
5.5
0.25
5.75
5.0
4.0
8.0
2.5
1.25
5.0
4.5
4.25
5.5
3.0
7.5
5.5
2.75
3.5
8.75
1.0
5.5
3.75
3.5
9.75
3.75
6.0
5.25
3.5
0.5
2.25
1.25
0.75
7.75
8.25
7.0
3.0
7.0
2.0
3.75
5.0
8.0
3.0
5.75
3.75
1.0
5.0
4.75
4.75
7.5
10.75
6.75
8.0
5.0
4.0
4.75
3.75
4.0
11.25
5.0
4.5
4.0
5.0
6.0
8.5
0.5
5.75
7.5
6.5
4.75
7.5
5.5
3.5
4.0
5.75
3.5
3.25
3.0
1.0
2.25
6.0
3.25
9.0
0.25
2.75
4.75
2.5
4.25
3.0
9.5
8.75
6.25
3.5
4.0
3.5
1.75
8.0
5.25
5.25
2.0
10.75
6.75
6.5
2.25
5.0
9.25
4.25
7.25
6.5
0.75
5.0
4.25
7.5
6.5
5.0
7.25
9.25
8.0
7.0
2.5
2.75
10.75
7.75
10.25
5.25
4.25
1.5
5.25
6.75
5.5
8.

In [309]:
data_matrix

array([[[[ 0.08618175,  0.08977199,  0.09279064, ..., -0.09984073,
           0.16442442, -0.13809787],
         [ 0.07163088,  0.07484148,  0.07758137, ..., -0.08336258,
           0.13634396, -0.1155228 ],
         [ 0.07564588,  0.07901096,  0.08247649, ..., -0.08889284,
           0.14488488, -0.12203915],
         ...,
         [-0.07659082, -0.07934121, -0.08319873, ...,  0.08885552,
          -0.14425949,  0.12253304],
         [-0.06777617, -0.07066835, -0.07380935, ...,  0.07853066,
          -0.13161029,  0.11094591],
         [-0.05934969, -0.06159058, -0.06467927, ...,  0.06964326,
          -0.11445949,  0.09717033]],

        [[ 0.10489235,  0.10931203,  0.11318198, ..., -0.12058695,
           0.19950629, -0.16814468],
         [ 0.08727947,  0.09124064,  0.09484103, ..., -0.10100089,
           0.16546752, -0.14070827],
         [ 0.0925356 ,  0.09669911,  0.10072692, ..., -0.10725205,
           0.17513745, -0.1477134 ],
         ...,
         [-0.09396097, -0.09727115

In [273]:
weights

array([10.25,  7.  ,  3.25, ...,  9.5 ,  3.5 ,  0.75])

In [324]:
s = np.zeros(len(service_features))
for i in range(len(service_features)):
    s[i] = data_matrix[i, :, :, :].sum()#заряд каждого словосочетания

t = np.zeros(len(tech_features))
for i in range(len(tech_features)):
    t[i] = data_matrix[:, i, :, :].sum()

d = np.zeros(len(driving_features))
for i in range(len(driving_features)):
    d[i] = data_matrix[:, :, i, :].sum()

b = np.zeros(len(body_features))
for i in range(len(body_features)):
    b[i] = data_matrix[:, :, :, i].sum()
#вычитаю случайное взаимодействие, результат модулярность
modularity_matrix = np.zeros(data_matrix.shape)
sum_pos_data_matrix = data_matrix.sum()**3
for si in range(len(service_features)):
    for ti in range(len(tech_features)):
        for di in range(len(driving_features)):
            for bi in range(len(body_features)):
                #проивзведение - это вес зарядов, но идет неправильная размерность, поэтому делим
                #на куб.
                #вычитаем потому что есть случайное взаимодействие. Так убираем шум
                modularity_matrix[si, ti, di, bi] = s[si]*t[ti]*d[di]*b[bi]/sum_pos_data_matrix
    

In [311]:
data_matrix_old = copy.deepcopy(data_matrix)

In [325]:
data_matrix1 = data_matrix - modularity_matrix

In [228]:
data_matrix.sum()/data_matrix.size

0.33186472969754705

### Теперь попробуем запустить на данной матрице нашу программу кластеризации

In [315]:
class Box:
    def __init__(self, data_matrix, initial_i):
        self.dim = len(data_matrix.shape)
        self.data = data_matrix
        self.lambda_0 = 0#так как мы получили модулярную матрицу, убрав шум
        self.data_shift = self.data - self.lambda_0 
        self.initialize_sets(initial_i)
    
    def initialize_sets(self, initial_i):
        self.sets = [set() for i in range(self.dim)]
        self.sets[0].add(initial_i)
        idx = []
        idx.append([initial_i])
        for i in range(1, self.dim):
            idx.append(slice(None))
        non_zero_args = np.nonzero(self.data[idx])
        for dim in range(1, self.dim):
            self.sets[dim] = set(non_zero_args[dim].tolist())
    
    def update(self, dif):
        if dif['inside']:
            self.sets[dif['dim']].remove(dif['entity'])
        else:
            self.sets[dif['dim']].add(dif['entity'])
            
    def try_update(self, dif):
        sets = copy.deepcopy(self.sets)
        if dif['inside']:
            sets[dif['dim']].remove(dif['entity'])
        else:
            sets[dif['dim']].add(dif['entity'])
        box_hash = self.get_hash(sets)
        return box_hash
    
    def get_cur_hash(self):
        return self.get_hash(self.sets)
    
    def get_hash(self, sets):
        common_list = []
        for cur_set in sets:
            common_list.extend(sorted(cur_set))
        box_hash = hash(tuple(common_list))
        return box_hash

class Clusterization:
    
    def get_clusters(self, data_matrix, euristic=0):
        boxes = []
        data_matrix_cur = copy.deepcopy(data_matrix)
        not_used_idx = [i for i in range(0, data_matrix.shape[0])]
        for i in range(data_matrix.shape[0]):
            ###### euristic data_matrix change
            if euristic == 1: #1st dim not overlapping 
                if i not in not_used_idx:
                    continue
            ################
            cur_box = self.FourClusterBox(i, data_matrix_cur, not_used_idx)
            new_box = True
            for box in boxes:
                if cur_box.sets == box.sets:
                    new_box = False
                    break
            for box_set in cur_box.sets:
                if len(box_set) == 0:
                    new_box = False
            if new_box:
                boxes.append(cur_box)
                ###### euristic 1
                if euristic == 1:
                    not_used_idx_upd = []
                    for idx in not_used_idx:
                        if idx not in cur_box.sets[0]:
                            not_used_idx_upd.append(idx)
                    not_used_idx = not_used_idx_upd
                #####################################
        return boxes

    def find_dif(self, box, dim, entity, inside):
        def get_z(inside):
            if inside:
                return -1
            return 1
        
        def get_cur_sets_sum(box):
            idx = []
            for box_set in box.sets:
                idx.append(list(box_set))
            return box.data_shift[np.ix_(*idx)].sum()
        
        def get_entity_sets_sum(box, dim, entity):
            idx = []
            for i, box_set in enumerate(box.sets):
                if i == dim:
                    idx.append([entity])
                else:
                    idx.append(list(box_set))
            return box.data_shift[np.ix_(*idx)].sum()
        
        z = get_z(inside)
        cur_sets_sum = get_cur_sets_sum(box)
        entity_sets_sum = get_entity_sets_sum(box, dim, entity)
        terms = []
        terms.append(entity_sets_sum**2)
        if box.dim == 2:
            terms[0] = terms[0] * z
        terms.append(2 * z * cur_sets_sum * entity_sets_sum)
        terms.append(((-1) * cur_sets_sum **2)/len(box.sets[dim]))
        if box.dim != 2:
            terms[2] = terms[2] * z
        numerator = np.array(terms).sum()
        set_lengths = 1
        for i in range(box.dim):
            if i != dim:
                set_lengths *= len(box.sets[i])
        denominator = (len(box.sets[dim]) + z) * set_lengths
        if denominator != 0:
            dif = numerator/denominator
        else:
            dif = -np.inf
        return dif

    def FourClusterBox(self, initial_i, data_matrix, not_used_idx):
        box = Box(data_matrix, initial_i)
        ind = 0
        start = time.time()
        uniq_boxes = set()#avoid repeating boxes
        uniq_boxes.add(box.get_cur_hash())
        while(True):
            ind += 1
            max_dif = {'value': -np.inf, 'dim': -1, 'entity': -1, 'inside': -1}
            for dim in range(box.dim):
                for entity in range(box.data.shape[dim]):
                    ########################## euristic add
                    if dim == 0 and entity not in not_used_idx:
                        continue
                    ################################
                    inside = entity in box.sets[dim]
                    if inside and len(box.sets[dim]) == 1:
                        continue
                    possible_dif = {'dim': dim, 'entity': entity, 'inside': inside}
                    cur_hash = box.try_update(possible_dif)
                    if cur_hash in uniq_boxes:#avoid box repeating
                        continue 
                    cur_dif = self.find_dif(box, dim, entity, inside)
                    #print('dif', cur_dif, 'sets', box.sets, 'dim', dim, 'entity', entity, inside)
                    if cur_dif > max_dif['value']:
                        max_dif['value'] = cur_dif
                        max_dif['dim'] = dim
                        max_dif['entity'] = entity
                        max_dif['inside'] = inside
            if max_dif['value'] <= 0:
                break
            box.update(max_dif)
            uniq_boxes.add(box.get_cur_hash())
        return box

In [327]:
%%time
clusters = Clusterization().get_clusters(data_matrix1, 0)

/Users/denissurin/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:17: FutureWarning: Using a non-tuple sequence for multidimensional indexing is deprecated; use `arr[tuple(seq)]` instead of `arr[seq]`. In the future this will be interpreted as an array index, `arr[np.array(seq)]`, which will result either in an error or a different result.


CPU times: user 17.1 s, sys: 226 ms, total: 17.3 s
Wall time: 17.5 s


In [328]:
clusters = [clusters[i].sets for i in range(len(clusters))]

In [329]:
rel_m = [relevance_matrix_service, relevance_matrix_tech, relevance_matrix_driving, 
         relevance_matrix_body]

In [330]:
len(clusters)

7

In [331]:
dfs=[]
types = ['service', 'tech', 'driving', 'body']
types_features = [service_features, tech_features, driving_features, 
                  body_features]
df = pd.DataFrame(columns=['service', 'tech', 'driving', 'body'])
for i_cluster, cluster in enumerate(clusters):
    for i in range(4):
        s = ''
        for j in list(cluster[i]):
            s+=(str(types_features[i][j]) + ' ,')
        df.loc[i_cluster, types[i]] = s
        print(s)
df

Детали не дорогие ,Дешевые детали ,Дешевые запчасти ,Недорогие запчасти ,Недорогое обслуживание ,Дешевое обслуживание ,
Двигатель надежный ,Двигатель экономичный ,Маленький расход ,
Отличное управление ,Легкое управление ,Неплохая управляемость ,Ураганная динамика ,Хорошая динамика ,Динамичное авто ,Машина очень быстрая ,Очень шустрая ,Динамика супер ,Динамики хватает ,Динамики за глаза ,
Машина не вместительная ,Машина не очень вместительная ,Не просторно ,Плохой обзор ,Слабый обзор ,Маленькие зеркала ,Немного места ,Маленький салон ,Не удобный салон ,Слабая шумоизоляция ,Маленький багажник ,
Детали не дорогие ,Дешевые детали ,Дешевые запчасти ,Недорогие запчасти ,Недорогое обслуживание ,Дешевое обслуживание ,
Двигатель мощный ,Двигатель надежный ,Двигатель экономичный ,Маленький расход ,
Отличное управление ,Легкое управление ,Неплохая управляемость ,Ураганная динамика ,Хорошая динамика ,Динамичное авто ,Машина очень быстрая ,Очень шустрая ,Динамика супер ,Динамики хватает ,Динамики 

,service,tech,driving,body
0,"Детали не дорогие ,Дешевые детали ,Дешевые зап...","Двигатель надежный ,Двигатель экономичный ,Мал...","Отличное управление ,Легкое управление ,Неплох...","Машина не вместительная ,Машина не очень вмест..."
1,"Детали не дорогие ,Дешевые детали ,Дешевые зап...","Двигатель мощный ,Двигатель надежный ,Двигател...","Отличное управление ,Легкое управление ,Неплох...","Машина вместительная ,Машина очень вместительн..."
2,"Детали не дорогие ,Дешевые детали ,Дешевые зап...","Двигатель не мощный ,Двигатель не надежный ,Дв...","Отличное управление ,Легкое управление ,Неплох...","Машина вместительная ,Машина очень вместительн..."
3,"Дорогие детали ,Недешевые детали ,Бракованные ...","Двигатель надежный ,Двигатель экономичный ,Мал...","Отличное управление ,Легкое управление ,Неплох...","Машина не вместительная ,Машина не очень вмест..."
4,"Дорогие детали ,Недешевые детали ,Бракованные ...","Двигатель мощный ,Двигатель надежный ,Двигател...","Отличное управление ,Легкое управление ,Неплох...","Машина вместительная ,Машина очень вместительн..."
5,"Недешевые детали ,Бракованные запчасти ,Недеше...","Двигатель не мощный ,Двигатель не надежный ,Дв...","Отличное управление ,Легкое управление ,Неплох...","Машина вместительная ,Машина очень вместительн..."
6,"Недорогое обслуживание ,Дешевое обслуживание ,","Двигатель не мощный ,Двигатель не надежный ,Дв...","Отличное управление ,Легкое управление ,Неплох...","Машина вместительная ,Машина очень вместительн..."


In [332]:
df.to_excel('car_clusters2.xlsx', index=False)